## Observations and Insights 

In [59]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
merge_df = pd.merge(study_results, mouse_metadata, on="Mouse ID", how="outer")

# Display the data table for preview
merge_df
#merge_df.isna().sum(axis=0)
#merge_df.count()
#mouse_metadata.count()
#study_results.count()

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,b128,0,45.000000,0,Capomulin,Female,9,22
1,b128,5,45.651331,0,Capomulin,Female,9,22
2,b128,10,43.270852,0,Capomulin,Female,9,22
3,b128,15,43.784893,0,Capomulin,Female,9,22
4,b128,20,42.731552,0,Capomulin,Female,9,22
...,...,...,...,...,...,...,...,...
1888,m601,25,33.118756,1,Capomulin,Male,22,17
1889,m601,30,31.758275,1,Capomulin,Male,22,17
1890,m601,35,30.834357,1,Capomulin,Male,22,17
1891,m601,40,31.378045,1,Capomulin,Male,22,17


In [60]:
# Checking the number of mice.
mice_count = len(merge_df["Mouse ID"].unique())
print(f"Number of mice included in the dataset: {mice_count}")

Number of mice included in the dataset: 249


In [61]:
# Check the data for ANY MOUSE ID WITH DUPLICATE TIME POINTS and REMOVE ANY DATA ASSOCIATED WITH THAT MOUSE ID.
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
merge_df[merge_df.duplicated(["Mouse ID", "Timepoint"], keep=False)]

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
860,g989,0,45.000000,0,Propriva,Female,21,26
861,g989,0,45.000000,0,Propriva,Female,21,26
862,g989,5,48.786801,0,Propriva,Female,21,26
863,g989,5,47.570392,0,Propriva,Female,21,26
864,g989,10,51.745156,0,Propriva,Female,21,26
865,g989,10,49.880528,0,Propriva,Female,21,26
866,g989,15,51.325852,1,Propriva,Female,21,26
867,g989,15,53.442020,0,Propriva,Female,21,26
868,g989,20,55.326122,1,Propriva,Female,21,26
869,g989,20,54.657650,1,Propriva,Female,21,26


In [62]:
# Optional: Get all the data for the duplicate mouse ID. 
duplicate_mouseID = merge_df.loc[merge_df["Mouse ID"] == "g989"]
duplicate_mouseID

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
860,g989,0,45.000000,0,Propriva,Female,21,26
861,g989,0,45.000000,0,Propriva,Female,21,26
862,g989,5,48.786801,0,Propriva,Female,21,26
863,g989,5,47.570392,0,Propriva,Female,21,26
864,g989,10,51.745156,0,Propriva,Female,21,26
865,g989,10,49.880528,0,Propriva,Female,21,26
866,g989,15,51.325852,1,Propriva,Female,21,26
867,g989,15,53.442020,0,Propriva,Female,21,26
868,g989,20,55.326122,1,Propriva,Female,21,26
869,g989,20,54.657650,1,Propriva,Female,21,26


In [63]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID. 
# stackoverflow resource: https://stackoverflow.com/questions/53182464/pandas-delete-a-row-in-a-dataframe-based-on-a-value
# first find indexes of all rows that's Mouse Id equals g989 as based on the instructions we are deleting any data related 
# to Mouse ID with duplicates timepoints 
# rows_to_delete = merge_df[merge_df["Mouse ID"] == "g989"].index
# rows_to_delete = duplicate_mouseID.index
# rows_to_delete
# clean_df = merge_df.drop(rows_to_delete)
# clean_df = merge_df.drop(merge_df["Mouse ID"].loc[merge_df["Mouse ID"] == "g989"].index)
clean_df = merge_df.drop(duplicate_mouseID.index)
clean_df

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,b128,0,45.000000,0,Capomulin,Female,9,22
1,b128,5,45.651331,0,Capomulin,Female,9,22
2,b128,10,43.270852,0,Capomulin,Female,9,22
3,b128,15,43.784893,0,Capomulin,Female,9,22
4,b128,20,42.731552,0,Capomulin,Female,9,22
...,...,...,...,...,...,...,...,...
1888,m601,25,33.118756,1,Capomulin,Male,22,17
1889,m601,30,31.758275,1,Capomulin,Male,22,17
1890,m601,35,30.834357,1,Capomulin,Male,22,17
1891,m601,40,31.378045,1,Capomulin,Male,22,17


In [64]:
# Checking the number of mice in the clean DataFrame.
mice_count_clean = len(clean_df["Mouse ID"].unique())
if mice_count_clean == mice_count:
    print(f"Not all duplicates were cleaned!!!")
else:
    print(f"Number of mice included in the cleaned dataset: {mice_count_clean}")

Number of mice included in the cleaned dataset: 248


## Summary Statistics

In [82]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
import numpy as np
from scipy.stats import sem

drug_regimens = clean_df["Drug Regimen"].unique()

# This method is the most straighforward, creating multiple series and putting them all together at the end.
means = [clean_df.loc[clean_df["Drug Regimen"] == i, :]["Tumor Volume (mm3)"].mean() for i in drug_regimens]
means2 = clean_df.groupby("Drug Regimen").mean()["Tumor Volume (mm3)"]
medians = [np.median(clean_df.loc[clean_df["Drug Regimen"] == i]["Tumor Volume (mm3)"]) for i in drug_regimens]
vars = [np.var(clean_df.loc[clean_df["Drug Regimen"] == i]["Tumor Volume (mm3)"]) for i in drug_regimens]
stds = [np.std(clean_df.loc[clean_df["Drug Regimen"] == i]["Tumor Volume (mm3)"]) for i in drug_regimens]
sems = [sem(clean_df.loc[clean_df["Drug Regimen"] == i]["Tumor Volume (mm3)"]) for i in drug_regimens]

tumor_volume_stats=pd.DataFrame({'Drug Regime': drug_regimens,
                        'Tumor Volume Mean': means,
                       'Tumor Volume Median': medians,
                       'Tumor Volume Var': vars,
                       'Tumor Volume Std': stds,
                       'Tumor Volume SEM': sems})
tumor_volume_stats.sort_values("Drug Regime")

,Drug Regime,Tumor Volume Mean,Tumor Volume Median,Tumor Volume Var,Tumor Volume Std,Tumor Volume SEM
0,Capomulin,40.675741,41.557809,24.839296,4.983904,0.329346
9,Ceftamin,52.591172,51.776157,39.069446,6.250556,0.469821
3,Infubinol,52.884795,51.820584,42.886388,6.548770,0.492236
1,Ketapril,55.235638,53.698743,68.188930,8.257659,0.603860
2,Naftisol,54.331565,52.509285,65.817708,8.112811,0.596466
8,Placebo,54.033581,52.288934,60.830138,7.799368,0.581331
7,Propriva,52.320930,50.446266,43.555716,6.599675,0.544332
5,Ramicane,40.216745,40.673236,23.383692,4.835669,0.320955
4,Stelasyn,54.233149,52.431737,59.122106,7.689090,0.573111
6,Zoniferol,53.236507,51.818479,48.266689,6.947423,0.516398


In [78]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# This method produces everything in a single groupby function
clean_df.groupby(["Drug Regimen"]).agg(['mean', 'median', 'var', 'std', 'sem'])["Tumor Volume (mm3)"]

,mean,median,var,std,sem
Drug Regimen,,,,,
Capomulin,40.675741,41.557809,24.947764,4.994774,0.329346
Ceftamin,52.591172,51.776157,39.290177,6.268188,0.469821
Infubinol,52.884795,51.820584,43.128684,6.567243,0.492236
Ketapril,55.235638,53.698743,68.553577,8.279709,0.603860
Naftisol,54.331565,52.509285,66.173479,8.134708,0.596466
Placebo,54.033581,52.288934,61.168083,7.821003,0.581331
Propriva,52.320930,50.446266,43.852013,6.622085,0.544332
Ramicane,40.216745,40.673236,23.486704,4.846308,0.320955
Stelasyn,54.233149,52.431737,59.450562,7.710419,0.573111


## Bar and Pie Charts

In [83]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pandas. 
drug_regimens = clean_df["Drug Regimen"].unique()
print(f"drug_regimens: {drug_regimens}")
mouse_counts=[]
for drug_regimen in drug_regimens:
    mouse_count=len(clean_df.loc[clean_df["Drug Regimen"] == drug_regimen, :]["Mouse ID"].unique())
    mouse_counts.append(mouse_count)
print(mouse_counts)

drug_regimens2 = clean_df.groupby(['Drug Regimen', 'Mouse ID'])
drug_regimens2['Drug Regimen'].count()

drug_regimens: ['Capomulin' 'Ketapril' 'Naftisol' 'Infubinol' 'Stelasyn' 'Ramicane'
 'Zoniferol' 'Propriva' 'Placebo' 'Ceftamin']
[25, 25, 25, 25, 24, 25, 25, 24, 25, 25]


Drug Regimen  Mouse ID
Capomulin     b128        10
              b742        10
              f966         5
              g288        10
              g316        10
                          ..
Zoniferol     q633        10
              s337        10
              w140         2
              w575        10
              x613         4
Name: Drug Regimen, Length: 248, dtype: int64

In [87]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pyplot.




In [11]:
# Generate a pie plot showing the distribution of female versus male mice using pandas



In [12]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot



## Quartiles, Outliers and Boxplots

In [13]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [14]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [15]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [16]:
# Generate a line plot of time point versus tumor volume for a mouse treated with Capomulin


In [17]:
# Generate a scatter plot of mouse weight versus average tumor volume for the Capomulin regimen


## Correlation and Regression

In [18]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
